### Build Docker Image that contains Resnet 152 model and Flask web application
Make sure you are able to run Docker without sudo.  
Make sure you are have logged in using docker login. 


In [1]:
import os
from os import path
import json

In [2]:
!rm -r flaskwebapp

In [3]:
!mkdir flaskwebapp
!mkdir flaskwebapp/nginx
!mkdir flaskwebapp/etc

Pull in Resnet 152 model

In [104]:
!wget http://download.tensorflow.org/models/resnet_v1_152_2016_08_28.tar.gz

--2018-03-19 18:02:18--  http://download.tensorflow.org/models/resnet_v1_152_2016_08_28.tar.gz
Resolving download.tensorflow.org (download.tensorflow.org)... 172.217.9.16, 2607:f8b0:4000:812::2010
Connecting to download.tensorflow.org (download.tensorflow.org)|172.217.9.16|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 224342140 (214M) [application/x-tar]
Saving to: ‘resnet_v1_152_2016_08_28.tar.gz.2’

resnet_v1_152_2016_ 100%[===================>] 213.95M   107MB/s    in 2.0s    

2018-03-19 18:02:20 (107 MB/s) - ‘resnet_v1_152_2016_08_28.tar.gz.2’ saved [224342140/224342140]



In [ ]:
!tar xvf resnet_v1_152_2016_08_28.tar.gz

Pull in class labels

In [4]:
!wget "http://data.dmlc.ml/mxnet/models/imagenet/synset.txt"

--2018-03-07 21:13:01--  http://data.dmlc.ml/mxnet/models/imagenet/synset.txt
Resolving data.dmlc.ml (data.dmlc.ml)... 54.208.175.7
Connecting to data.dmlc.ml (data.dmlc.ml)|54.208.175.7|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 31675 (31K) [text/plain]
Saving to: ‘synset.txt.1’

synset.txt.1        100%[===================>]  30.93K  --.-KB/s    in 0.03s   

2018-03-07 21:13:01 (888 KB/s) - ‘synset.txt.1’ saved [31675/31675]



In [4]:
!cp resnet_v1_152.ckpt flaskwebapp
!cp synset.txt flaskwebapp
!ls flaskwebapp

etc  nginx  resnet_v1_152.ckpt	synset.txt


Below is the driver for our model. The methods in this module will be called by our webapp.

In [5]:
%%writefile flaskwebapp/driver.py
import numpy as np
import logging, sys, json
import timeit as t
import base64
from PIL import Image, ImageOps
from io import BytesIO

import tensorflow as tf
from tensorflow.contrib.slim.nets import resnet_v1

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger("model_driver")

#TODO: Make these env variables
_MODEL_FILE = "resnet_v1_152.ckpt"
_LABEL_FILE = "synset.txt"
topResult = 3


def _create_label_lookup(label_path):
    with open(label_path, 'r') as f:
        label_list = [l.rstrip() for l in f]
        
    def _label_lookup(*label_locks):
        return [label_list[l] for l in label_locks]
    
    return _label_lookup


def _load_tf_model(checkpoint_file):
    # Placeholder
    input_tensor = tf.placeholder(tf.float32, shape=(None,224,224,3), name='input_image')
    
    # Load the model
    sess = tf.Session()
    arg_scope = resnet_v1.resnet_arg_scope()
    with tf.contrib.slim.arg_scope(arg_scope):
        logits, _ = resnet_v1.resnet_v1_152(input_tensor, num_classes=1000, is_training=False)
    probabilities = tf.nn.softmax(logits)
    
    saver = tf.train.Saver()
    saver.restore(sess, checkpoint_file)
    
    def predict_for(image):
        pred, pred_proba = sess.run([logits,probabilities], feed_dict={input_tensor: image})
        return pred_proba
    
    return predict_for


def _base64img_to_numpy(base64_img_string):
    if base64_img_string.startswith('b\''):
            base64_img_string = base64_img_string[2:-1]
    base64Img = base64_img_string.encode('utf-8')

    # Preprocess the input data 
    startPreprocess = t.default_timer()
    decoded_img = base64.b64decode(base64Img)
    img_buffer = BytesIO(decoded_img)

    # Load image with PIL (RGB)
    pil_img = Image.open(img_buffer).convert('RGB')
    pil_img = ImageOps.fit(pil_img, (224, 224), Image.ANTIALIAS)
    return np.array(pil_img, dtype=np.float32)


def create_scoring_func(model_path=_MODEL_FILE, label_path=_LABEL_FILE):
    start = t.default_timer()
    labels_for = _create_label_lookup(label_path)
    predict_for = _load_tf_model(model_path)
    end = t.default_timer()

    loadTimeMsg = "Model loading time: {0} ms".format(round((end-start)*1000, 2))
    logger.info(loadTimeMsg)
    
    def call_model(image_array, number_results=topResult):
        pred_proba = predict_for(image_array).squeeze()
        selected_results = np.flip(np.argsort(pred_proba), 0)[:number_results]
        labels = labels_for(*selected_results)
        return list(zip(labels, pred_proba[selected_results].astype(np.float64)))
    return call_model


def get_model_api():
    scoring_func = create_scoring_func()
    
    def process_and_score(inputString, number_results=topResult):
        start = t.default_timer()

        images = json.loads(inputString)
        result = []
        totalPreprocessTime = 0
        totalEvalTime = 0
        totalResultPrepTime = 0

        for base64_img_string in images:
            rgb_image = _base64img_to_numpy(base64_img_string)
            batch_image = np.expand_dims(rgb_image, 0)
            result = scoring_func(batch_image, number_results=topResult)
        
        end = t.default_timer()

        logger.info("Predictions: {0}".format(result))
        logger.info("Predictions took {0} ms".format(round((end-start)*1000, 2)))
#         logger.info("Time distribution: preprocess={0} ms, eval={1} ms, resultPrep = {2} ms".format(round(totalPreprocessTime * 1000, 2), round(totalEvalTime * 1000, 2), round(totalResultPrepTime * 1000, 2)))

#         actualWorkTime = round((totalPreprocessTime + totalEvalTime + totalResultPrepTime)*1000, 2)
        return (result, 'Computed in {0} ms'.format(round((end-start)*1000, 2)))
    return process_and_score

def version():
    return tf.__version__
    


Writing flaskwebapp/driver.py


In [6]:
%run flaskwebapp/driver.py

In [7]:
from testing_utilities import img_url_to_json

In [8]:
IMAGEURL = "https://www.britishairways.com/assets/images/information/about-ba/fleet-facts/airbus-380-800/photo-gallery/240x295-BA-A380-exterior-2-high-res.jpg"

In [9]:
jsonimg = img_url_to_json(IMAGEURL)

In [10]:
json_lod= json.loads(jsonimg)

In [11]:
predict_for = get_model_api()

INFO:tensorflow:Restoring parameters from resnet_v1_152.ckpt


INFO:tensorflow:Restoring parameters from resnet_v1_152.ckpt
INFO:model_driver:Model loading time: 12342.27 ms


In [14]:
output = predict_for(json_lod['input'])

INFO:model_driver:Predictions: [('n02690373 airliner', 0.58137083053588867), ('n04266014 space shuttle', 0.31709545850753784), ('n04592741 wing', 0.072090521454811096)]
INFO:model_driver:Predictions took 47.44 ms


In [15]:
json.dumps(output)

'[[["n02690373 airliner", 0.5813708305358887], ["n04266014 space shuttle", 0.31709545850753784], ["n04592741 wing", 0.0720905214548111]], "Computed in 47.44 ms"]'

Below is the module for the Flask web application.

In [16]:
%%writefile flaskwebapp/app.py
from flask import Flask, request
import time
import driver
import logging
import json

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

app = Flask(__name__)
predict_for = driver.get_model_api()

@app.route('/score', methods = ['POST'])
def scoreRRS():
    """ Endpoint for scoring
    """
    logger.debug('')
    if request.headers['Content-Type'] != 'application/json':
        return Response(json.dumps({}), status= 415, mimetype ='application/json')
    
    request_input = request.json['input']
    predictions = predict_for(request_input)
    return json.dumps({'result': predictions})


@app.route("/")
def healthy():
    return "Healthy"


@app.route('/version', methods = ['GET'])
def version_request():
    return driver.version()


if __name__ == "__main__":
    app.run(host='0.0.0.0') # Ignore, Development server

Writing flaskwebapp/app.py


In [17]:
%%writefile flaskwebapp/wsgi.py
import sys
sys.path.append('/code/') # FIXME: This is horrible
from app import app as application

def create():
    print("Initialising")
    application.run(host='127.0.0.1', port=5000)

Writing flaskwebapp/wsgi.py


In [18]:
%%writefile flaskwebapp/requirements.txt
pillow
click==6.7
configparser==3.5.0
Flask==0.11.1
gunicorn==19.6.0
json-logging-py==0.2
MarkupSafe==1.0
olefile==0.44
requests==2.12.3

Writing flaskwebapp/requirements.txt


The configuration for the Nginx. Note that it creates a proxy between ports **88** and **5000**.

In [19]:
%%writefile flaskwebapp/nginx/app
server {
    listen 80;
    server_name _;
 
    location / {
    include proxy_params;
    proxy_pass http://127.0.0.1:5000;
    proxy_connect_timeout 5000s;
    proxy_read_timeout 5000s;
  }
}

Writing flaskwebapp/nginx/app


In [20]:
image_name = "masalvar/tfresnet-gpu"
application_path = 'flaskwebapp'
docker_file_location = path.join(application_path, 'dockerfile')

In [21]:
%%writefile flaskwebapp/gunicorn_logging.conf

[loggers]
keys=root, gunicorn.error

[handlers]
keys=console

[formatters]
keys=json

[logger_root]
level=INFO
handlers=console

[logger_gunicorn.error]
level=ERROR
handlers=console
propagate=0
qualname=gunicorn.error

[handler_console]
class=StreamHandler
formatter=json
args=(sys.stdout, )

[formatter_json]
class=jsonlogging.JSONFormatter

Writing flaskwebapp/gunicorn_logging.conf


In [22]:
%%writefile flaskwebapp/kill_supervisor.py
import sys
import os
import signal


def write_stdout(s):
    sys.stdout.write(s)
    sys.stdout.flush()

# this function is modified from the code and knowledge found here: http://supervisord.org/events.html#example-event-listener-implementation
def main():
    while 1:
        write_stdout('READY\n')
        # wait for the event on stdin that supervisord will send
        line = sys.stdin.readline()
        write_stdout('Killing supervisor with this event: ' + line);
        try:
            # supervisord writes its pid to its file from which we read it here, see supervisord.conf
            pidfile = open('/tmp/supervisord.pid','r')
            pid = int(pidfile.readline());
            os.kill(pid, signal.SIGQUIT)
        except Exception as e:
            write_stdout('Could not kill supervisor: ' + e.strerror + '\n')
            write_stdout('RESULT 2\nOK')

main()


Writing flaskwebapp/kill_supervisor.py


In [23]:
%%writefile flaskwebapp/etc/supervisord.conf 
[supervisord]
logfile=/tmp/supervisord.log ; (main log file;default $CWD/supervisord.log)
logfile_maxbytes=50MB        ; (max main logfile bytes b4 rotation;default 50MB)
logfile_backups=10           ; (num of main logfile rotation backups;default 10)
loglevel=info                ; (log level;default info; others: debug,warn,trace)
pidfile=/tmp/supervisord.pid ; (supervisord pidfile;default supervisord.pid)
nodaemon=true               ; (start in foreground if true;default false)
minfds=1024                  ; (min. avail startup file descriptors;default 1024)
minprocs=200                 ; (min. avail process descriptors;default 200)

[program:gunicorn]
command=bash -c "gunicorn --workers 1 -m 007 --timeout 100000 --capture-output --error-logfile - --log-level debug --log-config gunicorn_logging.conf \"wsgi:create()\""
directory=/code
redirect_stderr=true
stdout_logfile =/dev/stdout
stdout_logfile_maxbytes=0
startretries=2
startsecs=20

[program:nginx]
command=/usr/sbin/nginx -g "daemon off;"
startretries=2
startsecs=5
priority=3

[eventlistener:program_exit]
command=python kill_supervisor.py
directory=/code
events=PROCESS_STATE_FATAL
priority=2

Writing flaskwebapp/etc/supervisord.conf


We create a custom image based on Ubuntu 16.04 and install all the necessary dependencies. This is in order to try and keep the size of the image as small as possible.

In [24]:
%%writefile flaskwebapp/dockerfile

FROM nvidia/cuda:8.0-cudnn6-devel-ubuntu16.04
MAINTAINER Mathew Salvaris <mathew.salvaris@microsoft.com>

RUN echo "deb http://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1604/x86_64 /" > /etc/apt/sources.list.d/nvidia-ml.list

RUN mkdir /code
WORKDIR /code
ADD . /code/
ADD etc /etc

RUN apt-get update && apt-get install -y --no-install-recommends \
        build-essential \
        ca-certificates \
        cmake \
        curl \
        git \
        nginx \
        supervisor \
        wget && \
        rm -rf /var/lib/apt/lists/*

ENV PYTHON_VERSION=3.5
RUN curl -o ~/miniconda.sh -O  https://repo.continuum.io/miniconda/Miniconda3-latest-Linux-x86_64.sh  && \
    chmod +x ~/miniconda.sh && \
    ~/miniconda.sh -b -p /opt/conda && \
    rm ~/miniconda.sh && \
    /opt/conda/bin/conda create -y --name py$PYTHON_VERSION python=$PYTHON_VERSION numpy scipy pandas scikit-learn && \
    /opt/conda/bin/conda clean -ya
ENV PATH /opt/conda/envs/py$PYTHON_VERSION/bin:$PATH
ENV LD_LIBRARY_PATH /opt/conda/envs/py$PYTHON_VERSION/lib:/usr/local/cuda/lib64/:$LD_LIBRARY_PATH

RUN rm /etc/nginx/sites-enabled/default && \
    cp /code/nginx/app /etc/nginx/sites-available/ && \
    ln -s /etc/nginx/sites-available/app /etc/nginx/sites-enabled/ && \
    pip install tensorflow-gpu==1.4.1 && \
    pip install -r /code/requirements.txt

EXPOSE 80
CMD ["supervisord", "-c", "/etc/supervisord.conf"]

Writing flaskwebapp/dockerfile


In [25]:
!docker build -t $image_name -f $docker_file_location $application_path

Sending build context to Docker daemon 241.6 MB
Step 1/15 : FROM nvidia/cuda:8.0-cudnn6-devel-ubuntu16.04
 ---> 547cf50ecba4
Step 2/15 : MAINTAINER Mathew Salvaris <mathew.salvaris@microsoft.com>
 ---> Using cache
 ---> cc7db6ee5d77
Step 3/15 : RUN echo "deb http://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1604/x86_64 /" > /etc/apt/sources.list.d/nvidia-ml.list
 ---> Using cache
 ---> 779253273a37
Step 4/15 : RUN mkdir /code
 ---> Using cache
 ---> 1b4fe14cc258
Step 5/15 : WORKDIR /code
 ---> Using cache
 ---> 3089fcef8872
Step 6/15 : ADD . /code/
 ---> d6ad94639481
Removing intermediate container f60a323b87f5
Step 7/15 : ADD etc /etc
 ---> 1cc66e4cb875
Removing intermediate container d9d1b0fd7711
Step 8/15 : RUN apt-get update && apt-get install -y --no-install-recommends         build-essential         ca-certificates         cmake         curl         git         nginx         supervisor         wget &&         rm -rf /var/lib/apt/lists/*
 ---> Running in ee

Get:24 http://archive.ubuntu.com/ubuntu xenial-updates/main amd64 libhx509-5-heimdal amd64 1.7~git20150920+dfsg-4ubuntu1.16.04.1 [107 kB]
Get:25 http://archive.ubuntu.com/ubuntu xenial/main amd64 libsqlite3-0 amd64 3.11.0-1ubuntu1 [396 kB]
Get:26 http://archive.ubuntu.com/ubuntu xenial-updates/main amd64 libkrb5-26-heimdal amd64 1.7~git20150920+dfsg-4ubuntu1.16.04.1 [202 kB]
Get:27 http://archive.ubuntu.com/ubuntu xenial-updates/main amd64 libheimntlm0-heimdal amd64 1.7~git20150920+dfsg-4ubuntu1.16.04.1 [15.1 kB]
Get:28 http://archive.ubuntu.com/ubuntu xenial-updates/main amd64 libgssapi3-heimdal amd64 1.7~git20150920+dfsg-4ubuntu1.16.04.1 [96.1 kB]
Get:29 http://archive.ubuntu.com/ubuntu xenial/main amd64 libsasl2-modules-db amd64 2.1.26.dfsg1-14build1 [14.5 kB]
Get:30 http://archive.ubuntu.com/ubuntu xenial/main amd64 libsasl2-2 amd64 2.1.26.dfsg1-14build1 [48.7 kB]
Get:31 http://archive.ubuntu.com/ubuntu xenial-updates/main amd64 libldap-2.4-2 amd64 2.4.42+dfsg-2ubuntu3.2 [160 kB]
G

Selecting previously unselected package libidn11:amd64.
Preparing to unpack .../libidn11_1.32-3ubuntu1.2_amd64.deb ...
Unpacking libidn11:amd64 (1.32-3ubuntu1.2) ...
Selecting previously unselected package libhogweed4:amd64.
Preparing to unpack .../libhogweed4_3.2-1ubuntu0.16.04.1_amd64.deb ...
Unpacking libhogweed4:amd64 (3.2-1ubuntu0.16.04.1) ...
Selecting previously unselected package libffi6:amd64.
Preparing to unpack .../libffi6_3.2.1-4_amd64.deb ...
Unpacking libffi6:amd64 (3.2.1-4) ...
Selecting previously unselected package libp11-kit0:amd64.
Preparing to unpack .../libp11-kit0_0.23.2-5~ubuntu16.04.1_amd64.deb ...
Unpacking libp11-kit0:amd64 (0.23.2-5~ubuntu16.04.1) ...
Selecting previously unselected package libtasn1-6:amd64.
Preparing to unpack .../libtasn1-6_4.7-3ubuntu0.16.04.3_amd64.deb ...
Unpacking libtasn1-6:amd64 (4.7-3ubuntu0.16.04.3) ...
Selecting previously unselected package libgnutls30:amd64.
Preparing to unpack .../libgnutls30_3.4.10-4ubuntu1.4_amd64.deb ...
Unpa

Selecting previously unselected package libxcb1:amd64.
Preparing to unpack .../libxcb1_1.11.1-1ubuntu1_amd64.deb ...
Unpacking libxcb1:amd64 (1.11.1-1ubuntu1) ...
Selecting previously unselected package libx11-data.
Preparing to unpack .../libx11-data_2%3a1.6.3-1ubuntu2_all.deb ...
Unpacking libx11-data (2:1.6.3-1ubuntu2) ...
Selecting previously unselected package libx11-6:amd64.
Preparing to unpack .../libx11-6_2%3a1.6.3-1ubuntu2_amd64.deb ...
Unpacking libx11-6:amd64 (2:1.6.3-1ubuntu2) ...
Selecting previously unselected package wget.
Preparing to unpack .../wget_1.17.1-1ubuntu1.3_amd64.deb ...
Unpacking wget (1.17.1-1ubuntu1.3) ...
Selecting previously unselected package curl.
Preparing to unpack .../curl_7.47.0-1ubuntu2.7_amd64.deb ...
Unpacking curl (7.47.0-1ubuntu2.7) ...
Selecting previously unselected package fonts-dejavu-core.
Preparing to unpack .../fonts-dejavu-core_2.35-1_all.deb ...
Unpacking fonts-dejavu-core (2.35-1) ...
Selecting previously unselected package fontconfi

Setting up libxslt1.1:amd64 (1.1.28-2.1ubuntu0.1) ...
Setting up nginx-common (1.10.3-0ubuntu0.16.04.2) ...
debconf: unable to initialize frontend: Dialog
debconf: (TERM is not set, so the dialog frontend is not usable.)
debconf: falling back to frontend: Readline
Setting up nginx-core (1.10.3-0ubuntu0.16.04.2) ...
invoke-rc.d: could not determine current runlevel
invoke-rc.d: policy-rc.d denied execution of start.
Setting up nginx (1.10.3-0ubuntu0.16.04.2) ...
Setting up python-pkg-resources (20.7.0-1) ...
Setting up python-meld3 (1.0.2-2) ...
Setting up supervisor (3.2.0-2ubuntu0.1) ...
invoke-rc.d: could not determine current runlevel
invoke-rc.d: policy-rc.d denied execution of start.
Processing triggers for libc-bin (2.23-0ubuntu10) ...
Processing triggers for ca-certificates (20170717~16.04.1) ...
Updating certificates in /etc/ssl/certs...
148 added, 0 removed; done.
Running hooks in /etc/ca-certificates/update.d...
done.
Processing triggers for systemd (229-4ubuntu21.1) ...
 ---

 ---> 1d3794493125
Removing intermediate container 11d069f9d486
Step 11/15 : ENV PATH /opt/conda/envs/py$PYTHON_VERSION/bin:$PATH
 ---> Running in fbf7d133e0d8
 ---> 23cbab64bd3b
Removing intermediate container fbf7d133e0d8
Step 12/15 : ENV LD_LIBRARY_PATH /opt/conda/envs/py$PYTHON_VERSION/lib:/usr/local/cuda/lib64/:$LD_LIBRARY_PATH
 ---> Running in f321d64620df
 ---> 99703a17cc13
Removing intermediate container f321d64620df
Step 13/15 : RUN rm /etc/nginx/sites-enabled/default &&     cp /code/nginx/app /etc/nginx/sites-available/ &&     ln -s /etc/nginx/sites-available/app /etc/nginx/sites-enabled/ &&     pip install tensorflow-gpu==1.4.1 &&     pip install -r /code/requirements.txt
 ---> Running in 9cb0b7ffc1d0
  Running setup.py bdist_wheel for html5lib: started
  Running setup.py bdist_wheel for html5lib: finished with status 'done'
  Stored in directory: /root/.cache/pip/wheels/6f/85/6c/56b8e1292c6214c4eb73b9dda50f53e8e977bf65989373c962
Successfully built html5lib
You are using pip

In [16]:
!docker push $image_name

The push refers to a repository [docker.io/masalvar/cntkresnet-gpu]

d670a9ff: Preparing 
197b748f: Preparing 
1ddff5fb: Preparing 
8863fbb1: Preparing 
fa8fbc32: Preparing 
6c6a5966: Preparing 
6c6a5966: Waiting g 
05e21031: Preparing 
0ba0a036: Preparing 
e48572eb: Preparing 
0ba0a036: Waiting g 
b03ecbb7: Preparing 
748c63e7: Waiting g 
4c622b50: Waiting g 
ea2bb533: Preparing 
89ea437e: Preparing 
8b9b1b5b: Preparing 
670a9ff: Pushing 1.347 GB/1.738 GB670a9ff: Pushed  1.739 GB/1.738 GBlatest: digest: sha256:00268c4c3d65f285aa9d0df846c3b5a223dfd64ec8286dacb95ddefaff8cb837 size: 4091


In [17]:
print('Docker image name {}'.format(image_name)) 

Docker image name masalvar/cntkresnet-gpu


### Test locally
Go to the [Test Locally notebook](TestLocally.ipynb) to test your Docker image